In [1]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

MODEL_NAME = "skt/kogpt2-base-v2"
MODEL_PATH = "./models"
SEQ_LEN = 50
TOKENS_DICT = {
    'additional_special_tokens':['<unused0>', '<unused1>'],
}

tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer.add_special_tokens(TOKENS_DICT)
print(tokenizer.special_tokens_map)

{'additional_special_tokens': "['<unused0>', '<unused1>']"}


In [2]:
# HuggingFace에서 top k와 top p로 함수 샘플링
import torch
import torch.nn.functional as F
from tqdm import trange


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):

    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # top-k의 마지막 토큰보다 확률이 낮은 모든 토큰을 제거
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # 임계값 이상의 누적 확률을 가진 토큰 제거
        sorted_indices_to_remove = cumulative_probs > top_p
        
        # 첫 번째 토큰도 임계값보다 높게 유지하려면 인덱스를 오른쪽으로 이동
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # 정렬된 텐서를 원래 인덱싱에 분산
        indices_to_remove = sorted_indices_to_remove.scatter(dim=1, index=sorted_indices, src=sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    return logits


# HuggingFace에서 컨텍스트/슬로건 분리 작업에 맞게 조정됨
def sample_sequence(model, length, context, segments_tokens=None, num_samples=1, temperature=1, top_k=0, top_p=0.0, repetition_penalty=1.0,
                    device='cpu'):
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context

    with torch.no_grad():
        for _ in trange(length):

            inputs = {'input_ids': generated}
            if segments_tokens != None:
              inputs['token_type_ids'] = torch.tensor(segments_tokens[:generated.shape[1]]).unsqueeze(0).repeat(num_samples, 1)


            outputs = model(**inputs)
            next_token_logits = outputs[0][:, -1, :] / (temperature if temperature > 0 else 1.)

            # CTRL의 반복 페널티(https://arxiv.org/abs/1909.05858)
            for i in range(num_samples):
                for _ in set(generated[i].tolist()):
                    next_token_logits[i, _] /= repetition_penalty
                
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            if temperature == 0: # greedy sampling
                next_token = torch.argmax(filtered_logits, dim=-1).unsqueeze(-1)
            else:
                next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token), dim=1)
    return generated


In [3]:
context = "드립 커피 전문점"

context_tkn = tokenizer.additional_special_tokens_ids[0]
slogan_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(context)

segments = [slogan_tkn] * SEQ_LEN
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [slogan_tkn]


epochs = [3, 6, 10]
for epoch in epochs:
  model.load_state_dict(torch.load(MODEL_PATH+ '/' + f'processed_slogan_{epoch}epoch_model.pth'))
  model.eval()

  # 최개길이 20의 20개의 슬로건 샘플
  # 확률분포를 조금 뾰족하게 하여 확률값이 높은 토큰이 살짝 더 잘나오도록 (temperature=0.9)
  # top_k 샘플링을 적용하여 확률값이 낮은 토큰들은 후보 단어에서 배제 (top_k=5)
  generated = sample_sequence(model, length=30, context=input_ids, segments_tokens=segments, temperature=0.7, top_k=40, top_p=0.8, num_samples=20)

  print('\n\n--- Generated Slogans ---\n')

  for g in generated:
    slogan = tokenizer.decode(g.squeeze().tolist())
    slogan = slogan.split('</s>')[0].split('<unused1>')[1]
    print(slogan)   

100%|██████████| 30/30 [00:28<00:00,  1.06it/s]




--- Generated Slogans ---

 한끼의 힘을 위해
 당신이 당신이 필요한 순간
 또 없던 가장 가까운 편안하게
 당신이 다른 세상
 한 번, 이제 우리 여행
 우리에게 더 좋은 여행한 모든 것
 건강한 여름의 시작
 지금도 함께하면 진짜 한끼의 시작
 우리만의 주일도 행복
 더 가까운 새로운 한 번째 당신이 봅니다
 한겨울 한끼, 다 없다
 한 번째 새로운 순간
 한끼의 시작
 또 필요한 새로운 세상을 위해
 당신이 다른 커피
 한 잔, 이제 함께하면 더 큰 대한민국
 대한민국, 한끼, 새로운 선택
 이제 좋은 모든 것을 넘어 힘
 지금도 행복
 좋은 순간


100%|██████████| 30/30 [00:28<00:00,  1.06it/s]




--- Generated Slogans ---

 내 스타일리이다
 여름은 커마로 짜릿하게
 세상에서 가장 가장 가장 가장 더 부드럽게
 여름은 더 더 좋은 것
 더 좋아하는 커피를 만나다
 오늘은 즐거움
 커피를 만나다
 이걸거보다 더 많은걸레는 한 알 수 없는 더 맛있는지
 내 커피를 넘어, 이 손 안에다
 이 여름은 한 번도 더 가까워집니다
 커피의 경계를 만나다
 오늘은 즐거움
 오늘은 커피를 넘어
 가장 좋아하는 카페, 이걸렘, 더 빛나는 순간
 오늘도 늘매일매일 짜릿하게
 여름은 쉬워라
 스타마피는 더 좋은 순간
 더 더 좋아하는 커피를 위한 새로운 방법
 커피의 상식을 채우다
 여름, 스타일이 되다


100%|██████████| 30/30 [00:28<00:00,  1.04it/s]



--- Generated Slogans ---

 커피의 창조남
 커피의 창조
 커피에 대한 순간, 필틱링의 시작
 커피를 입는 순간, 모든 순간
 우리의 커피를 위해 모든 순간, 이 곳을 담아
 커피에 대한 모든 순간
 커피에 커피 한잔의 창조 담아
 우리의 커피를 위한 가장 빛나는 순간
 커피의 창조봄, 모든 모든 것을 새롭게
 커피의 창조다
 모든 차원의 커피를 위한 커피
 커피를 입는 순간, 이기는 향합니다
 더 맛있는 순간
 이 곳을 즐기는 향
 커피에 대한 곳에
 커피의 창조 않는다
 커피의 창조
 커피의 창조
 커피의 모든 순간
 커피의 창조 채운다


In [5]:
for epoch in epochs:
  model.load_state_dict(torch.load(MODEL_PATH+ '/' + f'processed_slogan_{epoch}epoch_model.pth'))
  model.eval()

  # 최개길이 20의 20개의 슬로건 샘플
  # 확률분포를 조금 뾰족하게 하여 확률값이 높은 토큰이 살짝 더 잘나오도록 (temperature=0.9)
  # top_k 샘플링과 top_p 샘플링을 동시에 적용하여 확률값이 낮은 토큰들은 후보 단어에서 배제 (top_k=50, top_p=0.95)
  generated = sample_sequence(model, length=20, context=input_ids, segments_tokens=segments, temperature=0.9, top_k=20, top_p=0.95, num_samples=20)

  print('\n\n--- Generated Slogans ---\n')

  for g in generated:
    slogan = tokenizer.decode(g.squeeze().tolist())
    slogan = slogan.split('</s>')[0].split('<unused1>')[1]
    print(slogan)   
    

100%|██████████| 20/20 [00:14<00:00,  1.34it/s]




--- Generated Slogans ---

 모든 사람은은 당신의 쉐드
 나만의 세상
 올 한잔, 내 폰은 좋은 맛
 우리에 더 없던 행복합니다
 오늘은 쉐밍
 좋은 순간
 우리 가족부터 대한민국, 세쾌한일?
 새로운 겨울으로 더 가까운 순간
 대한민국 우리 나만의 새로운 길을 위해
 모든 사람은 넌 더 건강한 맛있는 기술
 당신이 필요한 당신이 챗
 한 번째 is me
 내 여름은 더 바르게 위해
 올일은 지금부터 좋은 순간
 올 세상, 당신이 다른 여행
 우리만의 세상이 있습니다. 우리 가족입니다
 이제, 당신이 다른 집합니다
 이제 올 여름은 당신과 함께
 SUE BLURPY BHE
 우리만의 당신이 더 나은 것을 위해


100%|██████████| 20/20 [00:15<00:00,  1.33it/s]




--- Generated Slogans ---

 이 커피의 새로운 시작
 스타마만의 일상을 바꾸다
 당신의 순간, 새로운 길을 찾아
 세상에 없던 쇼핑, 카페, 시작
 여름이 필요한 것을 한 조각이 됩니다
 여름이 되다, 맥주니까
 당신의 프리엄 시작은 더 큰 순간이 있다
 한잔이 온만의 레시피
 모든 순간, 여름
 여름은 더 좋아하는커피니까
 커피를 한잔를 만나다
 여름보다 쿨하게 가득의 새로운
 커피를 한잔의 선택
 오늘은 더 빛나는 모든 순간
 세상에 없던 카페
 오늘은 커피의 새로운 선택
 올 겨울에 더 완벽한 청정함을 만나다
 커피이 한잔이 바뀐다
 오늘은 시작은 즐거울수록 행복은 아니다
 상큼이 다른 세상


100%|██████████| 20/20 [00:14<00:00,  1.34it/s]



--- Generated Slogans ---

 커피에 다르게 하다
 커피의 여유를 채운다
 커피의 창조
 더 완벽한 프리미엄 커피브
 커피의 여유를 채운다
 우리의 행복이 커피는 놀라운 꽃
 커피에 모든 것을 다르게합니다
 커피의 모든 순간
 커피, 한 잔의 모든 것
 커피에 대한 기대를 위한 모든 순간
 커피에 대한 다르게 더 더 예쁘네
 다르게 즐기는 향
 당신의 차원의 커피를 새롭게
 커피를 부른다
 커피를 넘어 모든 경계를 위해 더 큰 옷을 위하여
 내 스타벅스의 커피를 힐링를 만나다
 내 노력의만의 차원의 커피를 부른다
 커피를 입는 순간도 화릿하게
 커피의 모든 도전이 새롭게 채운다
 커피에 필링의 모든 순간


In [6]:
context = "어쌔씬크리드 발할라, 액션 게임"

context_tkn = tokenizer.additional_special_tokens_ids[0]
slogan_tkn = tokenizer.additional_special_tokens_ids[1]

input_ids = [context_tkn] + tokenizer.encode(context)

segments = [slogan_tkn] * SEQ_LEN
segments[:len(input_ids)] = [context_tkn] * len(input_ids)

input_ids += [slogan_tkn]


epochs = [6, 10]
for epoch in epochs:
  model.load_state_dict(torch.load(MODEL_PATH+ '/' + f'processed_slogan_{epoch}epoch_model.pth'))
  model.eval()

  # 최개길이 20의 20개의 슬로건 샘플
  # 확률분포를 조금 뾰족하게 하여 확률값이 높은 토큰이 살짝 더 잘나오도록 (temperature=0.9)
  # top_k 샘플링을 적용하여 확률값이 낮은 토큰들은 후보 단어에서 배제 (top_k=5)
  generated = sample_sequence(model, length=30, context=input_ids, segments_tokens=segments, temperature=0.8, top_k=40, top_p=0.85, num_samples=20)

  print('\n\n--- Generated Slogans ---\n')

  for g in generated:
    slogan = tokenizer.decode(g.squeeze().tolist())
    slogan = slogan.split('</s>')[0].split('<unused1>')[1]
    print(slogan)   

100%|██████████| 30/30 [00:37<00:00,  1.25s/it]




--- Generated Slogans ---

 모든 플레이이 필요한 것
 모든 것형가 다른 모든 순간
 비짝간의 고상의 초미
 오늘은 아빠은 아니다
 RPG, SORT, WAUL를 가져라
 모바일와 감온 전감
 새로운 신세계
 새로운 플레이
 그 이상
 당신이 위해
 처음 만나는 즐거움
 너를 넘어, 전 플레이의 선택
 너만의 스타일로 만나는 모든 것
 더 만나는 즐거움
 모바일만의 야리의 선택
 이것은 초미먼지, 모든 순간
 모바일 골프를 플레이하라
 SIGHE THEAYE
 너를 넘어, 이제, 바로 플레이
 그 이상가 된다


100%|██████████| 30/30 [00:36<00:00,  1.23s/it]



--- Generated Slogans ---

 이제, 모든 곳이 만나는 소녀
 카바일로 이 위대한 가문전쟁
 진짜 스타일에 자유를 즐겨라
 다시, 모든것이다
 진짜 이제 모든 순간, 항상 연맹
 이 세상 모든 순간, 부딪쳐
 진짜 세상, 이 땅을 위한 스타일로
 레전볼루션의 시작과 판스, 시작
 내 방식대로 즐겨라
 이제, 모든 순간
 RePadelful your Chour space
 이제 내 손에서 시작
 비거도 스타일은 영웅, 시작은 역시
 지금, 스타일은 나만의 리그을 위해
 내 안의 도전이 플레이이 함께라면 스타릴 것
 이제 지금도 게임을 쥐는 스타일은 스타일은 재밌다
 내 방식대로 즐기는 진짜 가을
 내 몸의 팀은 이긴다
 모든 순간이 감던 타벨함을 즐겨봐
 이것은 내 몸을 이 번째 식
